In [1]:
# Read in input data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# contains info
import tensorflow.examples.tutorials.mnist.mnist as mnist_info

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
!ls MNIST_data/

t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [4]:
mnist_info.IMAGE_PIXELS

784

In [5]:
import tensorflow as tf
tf.reset_default_graph()
x = tf.placeholder(tf.float32,shape = [None,784]) 
# None, because we don't specify how many examples we'll look at

W = tf.Variable(tf.zeros([784, 10])) # number of weights
b = tf.Variable(tf.zeros([10])) # number of bias terms

In [6]:
y_hat = tf.nn.softmax(tf.matmul(x, W) + b) 

In [7]:
# correct answers

y = tf.placeholder(tf.float32, [None, 10])

In [8]:
ce = tf.reduce_mean(-tf.reduce_sum( y* tf.log(y_hat),reduction_indices=[1]))

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(ce)

In [10]:
sess = tf.Session()

In [11]:
sess.run(tf.global_variables_initializer())

In [12]:
# monitor accuracy
def acc():
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_hat,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))
loss_score = []
for i in range(1000):
    # get batches of training data
    # we don't show everything to the network at once
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
    if i%100==0:
        acc()

0.293
0.8761
0.8999
0.9093
0.9133
0.9161
0.9144
0.9165
0.9188
0.9212


In [13]:
acc()

0.9161


In [14]:
from graph_vis import show_graph as sg
sg(tf.get_default_graph())

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# Read input_data (not as one_hot)
from tensorflow.examples.tutorials.mnist import input_data

# new folder
mnist = input_data.read_data_sets("/tmp/data/")

# Assign them to values
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# define features
import tensorflow as tf
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

# dense neural network classifier
# three layers 300, 200 and 100
# 10 clases
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,200,100], n_classes=10,
                                         feature_columns=feature_cols)

# if TensorFlow >= 1.1, make compatible with sklearn
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f72a355f898>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmptgcgekyu'}


In [4]:
# fit the model, 4000 iterations
dnn_clf.fit(X_train, y_train, batch_size=50, steps=6000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmptgcgekyu/model.ckpt.
INFO:tensorflow:loss = 2.36699, step = 1
INFO:tensorflow:global_step/sec: 201.928
INFO:tensorflow:loss = 0.269074, step = 101 (0.497 sec)
INFO:tensorflow:global_step/sec: 209.245
INFO:tensorflow:loss = 0.265344, step = 201 (0.478 sec)
INFO:tensorflow:global_step/sec: 205.904
INFO:tensorflow:loss = 0.446199, step = 301 (0.486 sec)
INFO:tensorflow:global_step/sec: 219.261
INFO:tensorflow:loss = 0.193094, step = 401 (0.456 sec)
INFO:tensorflow:global_step/sec: 211.7
INFO:tensorflow:loss = 0.227325, step = 501 (0.472 sec)
INFO:tensorflow:global_step/

SKCompat()

In [5]:
# Calculate accuracies
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
print()
print('Accuracy',accuracy_score(y_test, y_pred['classes']))

INFO:tensorflow:Restoring parameters from /tmp/tmptgcgekyu/model.ckpt-6000

Accuracy 0.9818


In [6]:
# Define hyperparameters and input size

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 200
n_hidden3 = 100
n_outputs = 10
# Reset graph
tf.reset_default_graph()
# Placeholders for data (inputs and targets)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
keep_prob = tf.placeholder(tf.float32)

In [7]:
# Define neuron layers (ReLU in hidden layers)
# We'll take care of Softmax for output with loss function

def neuron_layer(X, n_neurons, name, activation=None):
    # X input to neuron
    # number of neurons for the layer
    # name of layer
    # pass in eventual activation function
    
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        
        # initialize weights to prevent vanishing / exploding gradients
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        
        # Initialize weights for the layer
        W = tf.Variable(init, name="kernel")
        # biases
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # Output from every neuron
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [8]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def leaky_relu(features, alpha=0.2, name=None):
  """Compute the Leaky ReLU activation function.

  "Rectifier Nonlinearities Improve Neural Network Acoustic Models"
  AL Maas, AY Hannun, AY Ng - Proc. ICML, 2013
  http://web.stanford.edu/~awni/papers/relu_hybrid_icml2013_final.pdf

  Args:
    features: A `Tensor` representing preactivation values.
    alpha: Slope of the activation function at x < 0.
    name: A name for the operation (optional).

  Returns:
    The activation value.
  """
  with ops.name_scope(name, "LeakyRelu", [features, alpha]):
    features = ops.convert_to_tensor(features, name="features")
    alpha = ops.convert_to_tensor(alpha, name="alpha")
    return math_ops.maximum(alpha * features, features)

In [9]:
# Define the hidden layers
import numpy as np
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=leaky_relu)
    drop_out1 = tf.nn.dropout(hidden1, keep_prob)
    hidden2 = neuron_layer(drop_out1, n_hidden2, name="hidden2",
                           activation=leaky_relu)
    drop_out2 = tf.nn.dropout(hidden2, keep_prob)
    hidden3 = neuron_layer(drop_out2, n_hidden3, name="hidden3",
                           activation=leaky_relu)
    drop_out3 = tf.nn.dropout(hidden3, keep_prob)
    logits = neuron_layer(drop_out3, n_outputs, name="outputs")

In [10]:
# Define loss function (that also optimizes Softmax for output):

with tf.name_scope("loss"):
    # logits are from the last output of the dnn
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [11]:
# Training step with Gradient Descent

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [12]:
# Evaluation to see accuracy

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [13]:
from graph_vis import show_graph as sg
sg(tf.get_default_graph())

In [14]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# change the iterations to 10000 to increase the accuracy 
n_epochs = 20
batch_size = 20

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, keep_prob: 0.9})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, keep_prob: 0.9})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                            y: mnist.validation.labels,
                                              keep_prob:1.0})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt") # save model

0 Train accuracy: 0.95 Val accuracy: 0.941
1 Train accuracy: 0.95 Val accuracy: 0.9546
2 Train accuracy: 1.0 Val accuracy: 0.9638
3 Train accuracy: 1.0 Val accuracy: 0.9654
4 Train accuracy: 1.0 Val accuracy: 0.9696
5 Train accuracy: 0.95 Val accuracy: 0.9714
6 Train accuracy: 1.0 Val accuracy: 0.9728
7 Train accuracy: 0.95 Val accuracy: 0.9746
8 Train accuracy: 1.0 Val accuracy: 0.975
9 Train accuracy: 1.0 Val accuracy: 0.9778
10 Train accuracy: 1.0 Val accuracy: 0.9774
11 Train accuracy: 0.9 Val accuracy: 0.9784
12 Train accuracy: 0.95 Val accuracy: 0.9782
13 Train accuracy: 1.0 Val accuracy: 0.9792
14 Train accuracy: 0.95 Val accuracy: 0.9802
15 Train accuracy: 0.95 Val accuracy: 0.9794
16 Train accuracy: 1.0 Val accuracy: 0.98
17 Train accuracy: 1.0 Val accuracy: 0.978
18 Train accuracy: 1.0 Val accuracy: 0.9804
19 Train accuracy: 1.0 Val accuracy: 0.9806


# Extra Credit part

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# Read input_data (not as one_hot)
from tensorflow.examples.tutorials.mnist import input_data

# new folder
mnist = input_data.read_data_sets("/tmp/data/")

# Assign them to values
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
from graph_vis import show_graph as sg
sg(tf.get_default_graph())

In [3]:
# define features
import tensorflow as tf
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

# dense neural network classifier
# three layers 400，300, 200 and 100
# 10 clases
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[500,400,300,200,100,50], n_classes=10,
                                         feature_columns=feature_cols)

# if TensorFlow >= 1.1, make compatible with sklearn
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d9a2daf98>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpmy_8lll0'}


In [4]:
# fit the model, 4000 iterations
dnn_clf.fit(X_train, y_train, batch_size=50, steps=6000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpmy_8lll0/model.ckpt.
INFO:tensorflow:loss = 2.28878, step = 1
INFO:tensorflow:global_step/sec: 113.965
INFO:tensorflow:loss = 0.685682, step = 101 (0.879 sec)
INFO:tensorflow:global_step/sec: 105.91
INFO:tensorflow:loss = 0.426951, step = 201 (0.944 sec)
INFO:tensorflow:global_step/sec: 94.2097
INFO:tensorflow:loss = 0.517173, step = 301 (1.062 sec)
INFO:tensorflow:global_step/sec: 91.2004
INFO:tensorflow:loss = 0.169976, step = 401 (1.097 sec)
INFO:tensorflow:global_step/sec: 103.972
INFO:tensorflow:loss = 0.155535, step = 501 (0.962 sec)
INFO:tensorflow:global_step

SKCompat()

In [5]:
# Calculate accuracies
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
print()
print('Accuracy',accuracy_score(y_test, y_pred['classes']))

INFO:tensorflow:Restoring parameters from /tmp/tmpmy_8lll0/model.ckpt-6000

Accuracy 0.981


In [23]:
# Define hyperparameters and input size

n_inputs = 28*28  # MNIST
n_hidden1 = 500
n_hidden2 = 400
n_hidden3 = 300
n_hidden4 = 200
n_hidden5 = 100
n_hidden6 = 50
n_outputs = 10
# Reset graph
tf.reset_default_graph()
# Placeholders for data (inputs and targets)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
keep_prob = tf.placeholder(tf.float32)

In [24]:
# Define neuron layers (ReLU in hidden layers)
# We'll take care of Softmax for output with loss function

def neuron_layer(X, n_neurons, name, activation=None):
    # X input to neuron
    # number of neurons for the layer
    # name of layer
    # pass in eventual activation function
    
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        
        # initialize weights to prevent vanishing / exploding gradients
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        
        # Initialize weights for the layer
        W = tf.Variable(init, name="kernel")
        # biases
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # Output from every neuron
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [25]:
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import candidate_sampling_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import sparse_ops
from tensorflow.python.ops import variables
def moments(x, axes,
            shift=None,  # pylint: disable=unused-argument
            name=None, keep_dims=False):

    with ops.name_scope(name, "moments", [x, axes]):
    # The dynamic range of fp16 is too limited to support the collection of
    # sufficient statistics. As a workaround we simply perform the operations
    # on 32-bit floats before converting the mean and variance back to fp16
       y = math_ops.cast(x, dtypes.float32) if x.dtype == dtypes.float16 else x
    # Compute true mean while keeping the dims for proper broadcasting.
       mean = math_ops.reduce_mean(y, axes, keep_dims=True, name="mean")
    # sample variance, not unbiased variance
       variance = math_ops.reduce_mean(
          math_ops.squared_difference(y, array_ops.stop_gradient(mean)),
           axes,
           keep_dims=True,
           name="variance")
    if not keep_dims:
            mean = array_ops.squeeze(mean, axes)
            variance = array_ops.squeeze(variance, axes)
    if x.dtype == dtypes.float16:
            return (math_ops.cast(mean, dtypes.float16), math_ops.cast(
              variance, dtypes.float16))
    else:
        return (mean, variance)

In [26]:
def batch_normalization(x,
                        mean,
                        variance,
                        offset,
                        scale,
                        variance_epsilon,
                        name=None):
  
  with ops.name_scope(name, "batchnorm", [x, mean, variance, scale, offset]):
    inv = math_ops.rsqrt(variance + variance_epsilon)
    if scale is not None:
        inv *= scale
    return x * inv + (offset - mean * inv
if offset is not None else -mean * inv)

In [27]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def leaky_relu(features, alpha=0.2, name=None):
    with ops.name_scope(name, "LeakyRelu", [features, alpha]):
        features = ops.convert_to_tensor(features, name="features")
        alpha = ops.convert_to_tensor(alpha, name="alpha")
        return math_ops.maximum(alpha * features, features)

In [28]:
# Define the hidden layers
#from keras.layers.normalization import BatchNormalization
import numpy as np
with tf.name_scope("dnn"):
    mean, variance = moments(X, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_X = batch_normalization(X,mean,variance,offset=True,scale=True,variance_epsilon=0.001,name=None) 
    hidden1 = neuron_layer(normalized_X, n_hidden1, name="hidden1",
                         activation=leaky_relu)
    drop_out1 = tf.nn.dropout(hidden1, keep_prob)
    mean1, variance1 = moments(drop_out1, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_1 = batch_normalization(drop_out1,mean1,variance1,offset=True,scale=True,variance_epsilon=0.001,name=None) 
    hidden2 = neuron_layer(normalized_1, n_hidden2, name="hidden2",
                           activation=leaky_relu)
    drop_out2 = tf.nn.dropout(hidden2, keep_prob)
    mean2, variance2 = moments(drop_out2, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_2 = batch_normalization(drop_out2,mean2,variance2,offset=True,scale=True,variance_epsilon=0.001,name=None) 
    hidden3 = neuron_layer(normalized_2, n_hidden3, name="hidden3",
                           activation=leaky_relu)
    drop_out3 = tf.nn.dropout(hidden3, keep_prob)
    mean3, variance3 = moments(drop_out3, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_3 = batch_normalization(drop_out3,mean3,variance3,offset=True,scale=True,variance_epsilon=0.001,name=None) 
    hidden4 = neuron_layer(normalized_3, n_hidden4, name="hidden4",
                           activation=leaky_relu)
    drop_out4 = tf.nn.dropout(hidden4, keep_prob)
    mean4, variance4 = moments(drop_out4, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_4 = batch_normalization(drop_out4,mean4,variance4,offset=True,scale=True,variance_epsilon=0.001,name=None)
    hidden5 = neuron_layer(normalized_4, n_hidden5, name="hidden4",
                           activation=leaky_relu)
    drop_out5 = tf.nn.dropout(hidden5, keep_prob)
    mean5, variance5 = moments(drop_out5, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_5 = batch_normalization(drop_out5,mean5,variance5,offset=True,scale=True,variance_epsilon=0.001,name=None)
    hidden6 = neuron_layer(normalized_5, n_hidden6, name="hidden4",
                           activation=leaky_relu)
    #drop_out5 = tf.nn.dropout(hidden5, keep_prob)
    # since the n_hidden6 is only 50, we can assume it won't cause overfitting
    mean6, variance6 = moments(hidden6, axes=[0],shift=None, name=None, keep_dims=False)
    normalized_6 = batch_normalization(hidden6,mean6,variance6,offset=True,scale=True,variance_epsilon=0.001,name=None)
    logits = neuron_layer(normalized_6, n_outputs, name="outputs")

In [35]:
# Define loss function (that also optimizes Softmax for output):

with tf.name_scope("loss"):
    # logits are from the last output of the dnn
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    for iteration in range(mnist.train.num_examples // batch_size):
        if iteration%10 ==0:
            print(loss)

Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:0", shape=(), dtype=float32)
Tensor("loss_2/loss:

In [36]:
# Training step with AdamOptimizer

learning_rate = 0.0001

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [37]:
# Evaluation to see accuracy

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [38]:
from graph_vis import show_graph as sg
sg(tf.get_default_graph())

In [32]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 30
batch_size = 40

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, keep_prob: 0.9})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, keep_prob: 0.9})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                            y: mnist.validation.labels,
                                              keep_prob: 1.0})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt") # save model

0 Train accuracy: 0.85 Val accuracy: 0.9248
1 Train accuracy: 0.95 Val accuracy: 0.9482
2 Train accuracy: 0.925 Val accuracy: 0.9556
3 Train accuracy: 0.925 Val accuracy: 0.9644
4 Train accuracy: 0.95 Val accuracy: 0.968
5 Train accuracy: 0.925 Val accuracy: 0.9682
6 Train accuracy: 0.975 Val accuracy: 0.9722
7 Train accuracy: 0.95 Val accuracy: 0.974
8 Train accuracy: 0.95 Val accuracy: 0.9778
9 Train accuracy: 0.95 Val accuracy: 0.9784
10 Train accuracy: 0.975 Val accuracy: 0.9788
11 Train accuracy: 0.95 Val accuracy: 0.9808
12 Train accuracy: 0.925 Val accuracy: 0.9818
13 Train accuracy: 1.0 Val accuracy: 0.9812
14 Train accuracy: 0.975 Val accuracy: 0.9806
15 Train accuracy: 0.975 Val accuracy: 0.9806
16 Train accuracy: 0.975 Val accuracy: 0.9804
17 Train accuracy: 0.95 Val accuracy: 0.9818
18 Train accuracy: 1.0 Val accuracy: 0.9806
19 Train accuracy: 0.975 Val accuracy: 0.9822
20 Train accuracy: 1.0 Val accuracy: 0.982
21 Train accuracy: 1.0 Val accuracy: 0.9826
22 Train accuracy